<a href="https://colab.research.google.com/github/SeanBarnier/HAFS_Air-Sea/blob/main/HAFSA_ocean-atmosphere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Set up environment

In [1]:
!pip install cfgrib
!pip install cartopy
!pip install tropycal

In [2]:
from tropycal import tracks, rain
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime as dt
import cfgrib
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cft

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


User parameters

In [4]:
name = "Milton"
tcNum = "14"
filepath = f"/content/drive/MyDrive/ColabNotebooks/{name}"
trackType = ""

centralTime = dt(year=2024, month=10, day=7, hour=6) #Time when Milton began its most rapid intensification
deltaDays = 2 #Days before and out from focal point

figureSuffix = "_RI"

Get best track data and find interested point

In [5]:
bt = pd.read_csv(filepath + "/hurdat2_" + name + trackType + ".csv")

centralTimeStr = centralTime.strftime("%Y-%m-%d %H:%M:%S")
point = (bt[bt.time == centralTimeStr].iloc[0].lat, bt[bt.time == centralTimeStr].iloc[0].lon)
point

(np.float64(22.1), np.float64(-92.9))

Retrieve HAFS-A Data

In [6]:
dateFormat = "%Y-%m-%d %H:%M:%S"
runFormat = "%Y%m%d%H"

start = centralTime - pd.Timedelta(days=deltaDays)
end = centralTime + pd.Timedelta(days=deltaDays)

"""start = start.strftime(dateFormat)
startDate, startTime = start.split(" ")
startYear, startMonth, startDay = startDate.split("-")
startHour, startMinute, startSecond = startTime.split(":")

end = tc.time[len(tc)-1]
endDT = dt.strptime(end, dateFormat)
endDate, endTime = end.split(" ")
endYear, endMonth, endDay = endDate.split("-")
endHour, endMinute, endSecond = endTime.split(":")"""

'start = start.strftime(dateFormat)\nstartDate, startTime = start.split(" ")\nstartYear, startMonth, startDay = startDate.split("-")\nstartHour, startMinute, startSecond = startTime.split(":")\n\nend = tc.time[len(tc)-1]\nendDT = dt.strptime(end, dateFormat)\nendDate, endTime = end.split(" ")\nendYear, endMonth, endDay = endDate.split("-")\nendHour, endMinute, endSecond = endTime.split(":")'

Find times needed

In [7]:
fcastTimes = {} #Key: initiation, item: Forecast Hour

initTime = start
while initTime <= end:
  validTime = initTime
  fcastTimes[initTime] = []

  while validTime <= end:
    fcastTimes[initTime].append(validTime)
    validTime += pd.Timedelta(hours=3)

  initTime += pd.Timedelta(hours=6)

"""for row in tc.iloc:
  fcastTimes[row.time] = []
  rowTime = dt.strptime(row.time, dateFormat)
  if rowTime.hour % 6 != 0 or rowTime.minute != 0: continue #Skip any lines that don't have a HAFS forecast at the same time

  for fhour in range(0, 127, 3):
    valid = rowTime + pd.Timedelta(hours=fhour)
    if valid <= endDT:
      fcastTimes[row.time].append(fhour)"""

"for row in tc.iloc:\n  fcastTimes[row.time] = []\n  rowTime = dt.strptime(row.time, dateFormat)\n  if rowTime.hour % 6 != 0 or rowTime.minute != 0: continue #Skip any lines that don't have a HAFS forecast at the same time\n\n  for fhour in range(0, 127, 3):\n    valid = rowTime + pd.Timedelta(hours=fhour)\n    if valid <= endDT:\n      fcastTimes[row.time].append(fhour)"

In [8]:
len(fcastTimes)

17

Get data from HAFS-A output.

In [8]:
#Data storage structure: Run time, valid time, layer, variable
amtVars = ["T", "q", "u", "v", "p"]
oceVars = ["T", "s", "u", "v"]

atmFiles = []
atm = {}
oceFiles = []
oce = {}

In [10]:
atmLayers = [925, 850, 700, 500] # in hPa
oceLayers = [30, 50, 100, 300] # In m. I don't know what these should be.

In [7]:
import xarray as xr
oceURL = "https://noaa-nws-hafs-pds.s3.amazonaws.com/hfsa/20241007/00/12l.2024100700.hfsa.mom6.f000.nc"
oceFile = "oce_20241007_00_f000.nc"
#!wget -O {oceFile} {oceURL}
oceData = xr.open_dataset(oceFile, decode_times=False)
oceData

<xarray.Dataset> Size: 2GB
Dimensions:    (xh: 2413, yh: 964, time: 1, xq: 2414, yq: 965, z_l: 55, z_i: 56)
Coordinates:
  * xh         (xh) float64 19kB -178.0 -177.9 -177.8 ... 14.8 14.88 14.96
  * yh         (yh) float64 8kB -23.03 -22.96 -22.89 ... 46.88 46.93 46.99
  * time       (time) float64 8B 0.1
  * xq         (xq) float64 19kB -178.0 -178.0 -177.9 ... 14.84 14.92 15.0
  * yq         (yq) float64 8kB -23.07 -23.0 -22.92 -22.85 ... 46.91 46.96 47.01
  * z_l        (z_l) float64 440B 1.0 3.5 7.5 ... 5.185e+03 5.395e+03 5.604e+03
  * z_i        (z_i) float64 448B 0.0 2.0 5.0 ... 5.29e+03 5.499e+03 5.708e+03
Data variables: (12/37)
    geolon     (yh, xh) float32 9MB ...
    geolat     (yh, xh) float32 9MB ...
    geolon_c   (yq, xq) float32 9MB ...
    geolat_c   (yq, xq) float32 9MB ...
    geolon_u   (yh, xq) float32 9MB ...
    geolat_u   (yh, xq) float32 9MB ...
    ...         ...
    evap       (time, yh, xh) float32 9MB ...
    lprec      (time, yh, xh) float32 9MB ...
    lrunoff    (time, yh, xh) float32 9MB ...
    fprec      (time, yh, xh) float32 9MB ...
    LwLatSens  (time, yh, xh) float32 9MB ...
    Heat_PmE   (time, yh, xh) float32 9MB ...
Attributes:
    NumFilesInSet:  1
    title:          ufs.hafs
    grid_type:      regular
    grid_tile:      N/A

In [ ]:
bucket = "https://noaa-nws-hafs-pds.s3.amazonaws.com/hfsa/"

for init, validList in fcastTimes.items():

  atm[init] = {}
  oce[init] = {}

  initDate, initTime = init.split(" ")
  initYear, initMonth, initDay = initDate.split("-")
  initHour, initMinute, initSecond = initTime.split(":")

  for valid in validList:

    atm[init][valid] = {}
    oce[init][valid] = {}

    fhour = str((valid-init).total_seconds() / 3600)
    while len(fhour) < 3: fhour = "0" + fhour

    atmURL = bucket + initDate.replace("-", "") + "/" + initHour + "/" + tcNum + "l." + initDate.replace("-", "") + initHour + ".hfsa.storm.atm.f" + fhour + ".grb2"
    atmFile = "atm_" + initDate.replace("-", "") + "_" + initHour + "_f" + fhour + ".grb2"
    atmFiles.append(atmFile)

    oceURL = bucket + initDate.replace("-", "") + "/" + initHour + "/" + tcNum + "l." + initDate.replace("-", "") + initHour + ".hfsa.mom6.f" + fhour + ".grb2"
    oceFile = "oce_" + initDate.replace("-", "") + "_" + initHour + "_f" + fhour + ".nc"
    oceFiles.append(oceFile)

    !wget -O {atmFile} {atmURL}
    !wget -O {oceFile} {oceURL}

    #_, dateStr, hour, fhourStr = atmFile.split("_")
    #runStr = dateStr + hour
    #if runStr not in atmMSLP.keys(): atmMSLP[runStr] = {}

    atmData = xr.open_dataset(atmFile, engine="cfgrib", decode_timedelta=True, filter_by_keys={'stepType': 'instant', 'typeOfLevel': 'layer'})
    oceData = xr.open_dataset(oceFile, decode_times=False)

    for layer in atmLayers:
      validPoint = atmData.sel(lat=point[0], lon=point[1], level=layer, method="closest")
      atm[init][valid][layer] = {}
      atm[init][valid][layer]["T"] = validPoint.temp.data
      atm[init][valid][layer]["q"] = validPoint.q.data #Not sure if these are the real variable names
      atm[init][valid][layer]["q"] = validPoint.p.data
      atm[init][valid][layer]["u"] = validPoint.u.data
      atm[init][valid][layer]["v"] = validPoint.v.data

    for layer in oceLayers:
      #validPoint = oceData.sel(lat=point[0], lon=point[1], level=layer, method="closest")
      oce[init][valid][layer] = {}
      oce[init][valid][layer]["T"] = oceData.sel(yh=point[0], xh=point[1], z_l=layer, method="closest").temp.data
      oce[init][valid][layer]["s"] = oceData.sel(yh=point[0], xh=point[1], z_l=layer, method="closest").so.data #Not sure if these are the real variable names
      oce[init][valid][layer]["u"] = oceData.sel(yh=point[0], xh=point[1], z_l=layer, method="closest").uo.data
      oce[init][valid][layer]["v"] = oceData.sel(yh=point[0], xh=point[1], z_l=layer, method="closest").vo.data

    !rm {atmFile}
    !rm {oceFile}